In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
mnist_data,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [4]:
mnist_train,mnist_test=mnist_data['train'],mnist_data['test']


In [5]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [6]:
num_validation_samples=0.1*(mnist_info.splits['train'].num_examples)
num_validation_samples=tf.cast(num_validation_samples,tf.int64)                 #to convert no. of samples to integer
num_validation_samples

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [7]:
num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

In [8]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)

scaled_test_data=mnist_test.map(scale)

# suffelling_data

In [14]:
buffer_size=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

batch_size=100
train_data=train_data.batch(batch_size)
validation_data=validation_data.batch(num_validation_samples)
test_data=scaled_test_data.batch(num_test_samples)

validation_inputs,validation_targets=next(iter(validation_data))

In [26]:
input_size=784
output_size=10
hidden_layer_size=100
model=tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [27]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# training

In [28]:
num_epochs=5
model.fit(train_data,epochs=num_epochs,validation_data=validation_data,verbose=2)

Epoch 1/5
540/540 - 14s - loss: 0.3316 - accuracy: 0.9056 - val_loss: 0.1841 - val_accuracy: 0.9473
Epoch 2/5
540/540 - 13s - loss: 0.1389 - accuracy: 0.9587 - val_loss: 0.1153 - val_accuracy: 0.9630
Epoch 3/5
540/540 - 13s - loss: 0.1009 - accuracy: 0.9701 - val_loss: 0.0973 - val_accuracy: 0.9723
Epoch 4/5
540/540 - 13s - loss: 0.0772 - accuracy: 0.9767 - val_loss: 0.0795 - val_accuracy: 0.9773
Epoch 5/5
540/540 - 13s - loss: 0.0627 - accuracy: 0.9814 - val_loss: 0.0735 - val_accuracy: 0.9770


In [29]:
testloss,test_accuracy=model.evaluate(test_data)

      1/Unknown - 2s 2s/step - loss: 0.0850 - accuracy: 0.9748